In [4]:
import pandas as pd
import numpy as np
import logging
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_data():
    try:
        data = fetch_california_housing(as_frame=True)
        df = data.frame
        logger.info("Data loaded successfully.")
        return df
    except Exception as e:
        logger.error("Failed to load dataset: %s", e)
        raise

def impute_data(df):
    try:
        imputer = SimpleImputer(strategy='mean')
        df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
        logger.info("Mean imputation successful.")
        return df_imputed
    except ValueError as e:
        logger.error("ValueError during imputation: %s", e)
        raise

def knn_impute_data(df):
    try:
        imputer = KNNImputer(n_neighbors=3)
        df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
        logger.info("KNN imputation successful.")
        return df_imputed
    except ValueError as e:
        logger.error("ValueError during KNN imputation: %s", e)
        raise

def scale_data(df):
    try:
        scaler = StandardScaler()
        scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
        logger.info("Standardization successful.")
        return scaled_df
    except Exception as e:
        logger.error("Error in standardization: %s", e)
        raise

def minmax_scale_data(df):
    try:
        scaler = MinMaxScaler()
        scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
        logger.info("Min-Max scaling successful.")
        return scaled_df
    except Exception as e:
        logger.error("Error in Min-Max scaling: %s", e)
        raise

def robust_scale_data(df):
    try:
        scaler = RobustScaler()
        scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
        logger.info("Robust scaling successful.")
        return scaled_df
    except Exception as e:
        logger.error("Error in robust scaling: %s", e)
        raise

def remove_highly_correlated(df, threshold=0.9):
    try:
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
        df_reduced = df.drop(columns=to_drop)
        logger.info("Highly correlated features removed: %s", to_drop)
        return df_reduced
    except Exception as e:
        logger.error("Error in correlation removal: %s", e)
        raise

def select_features_by_mi(X, y, top_k=5):
    try:
        mi = mutual_info_classif(X, y)
        selected_features = X.columns[np.argsort(mi)[-top_k:]]
        logger.info("Selected top mutual information features: %s", list(selected_features))
        return X[selected_features]
    except Exception as e:
        logger.error("Error in mutual information feature selection: %s", e)
        raise

def apply_variance_threshold(df, threshold=0.01):
    try:
        selector = VarianceThreshold(threshold=threshold)
        reduced_df = pd.DataFrame(selector.fit_transform(df), columns=df.columns[selector.get_support()])
        logger.info("Low variance features removed.")
        return reduced_df
    except Exception as e:
        logger.error("Error in variance thresholding: %s", e)
        raise

def main():
    df = load_data()
    df_missing = df.copy()
    df_missing.iloc[0:10, 0] = np.nan  # introduce missing values

    df_imputed = impute_data(df_missing)
    df_knn_imputed = knn_impute_data(df_missing)

    df_scaled = scale_data(df_imputed)
    df_minmax_scaled = minmax_scale_data(df_imputed)
    df_robust_scaled = robust_scale_data(df_imputed)

    df_uncorrelated = remove_highly_correlated(df_scaled)

    X = df_scaled.drop(columns=['MedHouseVal'])
    y = df_scaled['MedHouseVal'] > df_scaled['MedHouseVal'].median()
    X_mi = select_features_by_mi(X, y)
    X_vt = apply_variance_threshold(X)

if __name__ == "__main__":
    main()


INFO:__main__:Data loaded successfully.
INFO:__main__:Mean imputation successful.
INFO:__main__:KNN imputation successful.
INFO:__main__:Standardization successful.
INFO:__main__:Min-Max scaling successful.
INFO:__main__:Robust scaling successful.
INFO:__main__:Highly correlated features removed: ['Longitude']


INFO:__main__:Selected top mutual information features: ['AveOccup', 'AveRooms', 'Latitude', 'Longitude', 'MedInc']
INFO:__main__:Low variance features removed.
